# Tutorial, Part I: Candidate Extraction

In this example, we'll be writing an application to extract **chemical-induced-disease relationships** from Pubmed abstracts, as per the [BioCreative CDR Challenge](http://www.biocreative.org/resources/corpora/biocreative-v-cdr-corpus/).  At core, we will be constructing a model to classify _candidate chemical-disease (C-D) relation mentions_ as either true or false.  To do this, we first need a set of such candidates- in this notebook, we'll use `DDLite` utilities to extract these candidates.

_Note: We run automated tests on this tutorial to make sure that it is always up to date!  However, certain interactive components cannot currently be tested automatically, and will be skipped with if-then statements using the variable below:_

In [1]:
%load_ext autoreload
%autoreload 2
import os
AUTOMATED_TESTING = os.environ.get('TESTING') is not None

from snorkel import SnorkelSession
session = SnorkelSession()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Loading the Corpus

First, we will load and pre-process the corpus, storing it for convenience in a `Corpus` object

### Configuring a document parser

We'll start by defining a `DocParser` class to read in Pubmed abstracts from [Pubtator]([Pubtator](http://www.ncbi.nlm.nih.gov/CBBresearch/Lu/Demo/PubTator/index.cgi)), where they are stored along with "gold" (i.e. hand-annotated by experts) *chemical* and *disease mention* annotations. We'll use the `XMLDocParser` class, which allows us to use [XPath queries](https://en.wikipedia.org/wiki/XPath) to specify the relevant sections of the XML format.

_Note that we are newline-concatenating text from the title and abstract together for simplicity, but if we wanted to, we could easily extend the `DocParser` classes to preserve information about document structure._

In [2]:
from snorkel.parser import XMLDocParser
xml_parser = XMLDocParser(
    path='data/CDR_DevelopmentSet.xml',
    doc='.//document',
    text='.//passage/text/text()',
    id='.//id/text()',
    keep_xml_tree=True)

### Selecting a sentence parser

Next, we'll use an NLP preprocessing tool to split the `Document` objects into sentences, tokens, and provide annotations--part-of-speech tags, dependency parse structure, lemmatized word forms, etc.--for these sentences.  Here we use the default `SentenceParser` class.

In [3]:
from snorkel.parser import SentenceParser
sent_parser = SentenceParser()

### Pre-processing & loading the corpus

Finally, we'll put this all together using a `CorpusParser` object, which will execute the parsers and store the results as a `Corpus`:

In [4]:
from snorkel.parser import CorpusParser
cp = CorpusParser(xml_parser, sent_parser)
%time corpus = cp.parse_corpus(name='CDR Corpus')

CPU times: user 5.11 s, sys: 183 ms, total: 5.29 s
Wall time: 30.1 s


In [5]:
doc = corpus.documents[2]
print doc

Document('6436733', Corpus (CDR Corpus))


In [6]:
sent = doc.sentences[0]
print sent

Sentence(Document('6436733', Corpus (CDR Corpus)), 0, u'Acute changes of blood ammonia may predict short-term adverse effects of valproic acid.')


### Saving the corpus
We can persist the parsed corpus in Snorkel's database backend for later reuse:

In [7]:
session.add(corpus)
session.commit()

In [8]:
from snorkel.utils import get_as_dict

### Reloading the corpus
We can also reload the corpus from disk, skipping the above parsing if it has already been done:

In [9]:
from snorkel.models import Corpus
corpus = session.query(Corpus).filter(Corpus.name == 'CDR Corpus').one()
corpus

Corpus (CDR Corpus)

## Writing a basic candidate extractor

Next, we'll write a basic function to extract **candidate disease mentions** from the corpus.  For this first attempt, we'll just write a function that checks for matches against a list (or _"dictionary"_) of disease phrases, constructed using some pre-compiled ontologies ([UMLS](https://www.nlm.nih.gov/research/umls/), [ORDO](http://www.orphadata.org/cgi-bin/inc/ordo_orphanet.inc.php), [DOID](http://www.obofoundry.org/ontology/doid.html), [NCBI Diseases](http://www.ncbi.nlm.nih.gov/CBBresearch/Dogan/DISEASE/); see `tutorial/data/diseases.py`).

We'll do this using a `CandidateSpace` object--which defines the basic candidates we consider, in this case n-grams up to a certain length--and a `Matcher` object, which filters this candidate space down.

In [10]:
from load_dictionaries import load_disease_dictionary
from snorkel.candidates import Ngrams
from snorkel.matchers import DictionaryMatch

# Load the disease phrase dictionary
diseases = load_disease_dictionary()
print "Loaded %s disease phrases!" % len(diseases)

# Define a candidate space
ngrams = Ngrams(n_max=3)

# Define a matcher
matcher = DictionaryMatch(d=diseases, longest_match_only=False)

Loaded 507899 disease phrases!


Note that we set `longest_match_only=False`, which means that we _will_ consider subsequences of phrases that match our dictionary.

The `Ngrams` operator is applied over our `Sentence` objects and returns `Ngram` objects, and the `Matcher` then filters these, so we apply our operators over the sentences in the corpus, storing the results in a `Candidates` object for convenience:

In [12]:
from snorkel.candidates import CandidateExtractor
ce = CandidateExtractor(ngrams, matcher)
%time c = ce.extract(corpus.get_sentences(), 'all-2', session=session, parallelism=4)

AssertionError: A conflicting state is already present in the identity map for key (<class 'snorkel.models.context.Context'>, (502,))

In [ ]:
from snorkel.features import SpanFeaturizer
span_featurizer = SpanFeaturizer()

# Testing stuff

In [16]:
sent = corpus.get_sentences()[3]
sent

Sentence(Document(u'6794356', Corpus (CDR Corpus)), 3, u'Sixty-three percent of these infants had tricuspid valve involvement.')

In [23]:
unigrams = Ngrams(n_max=1)
list(unigrams.apply(sent))

[TemporarySpan("Sixty-three", context=505, chars=[0,10], words=[0,0]),
 TemporarySpan("percent", context=505, chars=[12,18], words=[1,1]),
 TemporarySpan("of", context=505, chars=[20,21], words=[2,2]),
 TemporarySpan("these", context=505, chars=[23,27], words=[3,3]),
 TemporarySpan("infants", context=505, chars=[29,35], words=[4,4]),
 TemporarySpan("had", context=505, chars=[37,39], words=[5,5]),
 TemporarySpan("tricuspid", context=505, chars=[41,49], words=[6,6]),
 TemporarySpan("valve", context=505, chars=[51,55], words=[7,7]),
 TemporarySpan("involvement", context=505, chars=[57,67], words=[8,8]),
 TemporarySpan(".", context=505, chars=[68,68], words=[9,9])]

# Testing featurization

In [ ]:
F = span_featurizer.fit_transform(c.candidates)

In [ ]:
[f for f in span_featurizer.inv_index.values() if 'None' in f]

### Saving the extracted candidates

In [ ]:
session.add(c)
session.commit()

### Reloading the candidates

In [ ]:
from snorkel.models import CandidateSet
c = session.query(CandidateSet).filter(CandidateSet.name == 'all').one()
c

In [ ]:
len(c)

In [ ]:
s1 = c.candidates[0]

In [ ]:
from snorkel.utils import get_as_dict

get_as_dict(s1.context)

## Evaluating our candidate recall on gold annotations

Next, we'll test our _candidate recall_--in other words, how many of the true disease mentions we picked up in our candidate set--using the gold annotations in our dataset.

The XML documents that we loaded using the `XMLDocParser` also contained annotations (this is why we kept the full xml tree using `keep_xml_tree=True`).  We'll load these annotations and map them to `Ngram` objects over our parsed sentences, that way we can easily compare our extracted candidate set with the gold annotations.  The code is fairly simple (see `tutorial/util.py`); note that we filter to only keep _disease_ annotations, and that the candidates should be uniquely identified by their `id` attribute:

In [ ]:
from utils import collect_pubtator_annotations
gold = []
for doc, sents in corpus:
    gold += [a for a in collect_pubtator_annotations(doc, sents) if a.meta['type'] == 'Disease']
gold = frozenset(gold)

In [ ]:
list(gold)[:5]

Now, we have a set of gold annotations of the same type as our candidates (`Ngram`), and can use set operations (where candidate objects are hashed by their `id` attribute), e.g.:

In [ ]:
len(gold.intersection(c.candidates))

For convenience, we'll use a basic helper method of the `Candidates` object:

In [ ]:
from snorkel.candidates import gold_stats
gold_stats(c, gold)

We note that our focus in this stage is on **acheiving high candidate recall, without considering an impractically large candidate set**.  Our main focus after this stage will be on training a classifier to select which candidates are true; this will raise precision while hopefully keeping recall high.  _Note however that candidate recall is an upper bound for the recall of this classifier!_

So, we have some work to do.

## Using the `Viewer` to inspect data

Next, we'll use the `Viewer` class--here, specifically, the `SentenceNgramViewer`--to inspect the data.

To start, we'll assemble a random set of all the sentences where there are gold annotations _not in our candidate set_, i.e. where we missed something, and then inspect these in the `Viewer`:

In [ ]:
from collections import defaultdict
from random import shuffle
from snorkel.models import Span

# Index the gold annotations by sentence id
gold_by_sid = defaultdict(list)
for g in gold:
    gold_by_sid[g.context.id].append(g)

# Get sentences

view_sents = [s for s in corpus.get_sentences() \
              if session.query(Span).filter(Span.context == s).count() \
                  < len(gold_by_sid[s.id])]
shuffle(view_sents)
view_sents = view_sents[:50]

Now, we instantiate and render the `Viewer` object; note we're being a bit sloppy, passing in _all_ the candidates and gold labels, but the `Viewer` object will take care of indexing them by sentence, and will only render the sentences we pass in:

In [ ]:
from snorkel.viewer import SentenceNgramViewer

# NOTE: This if-then statement is only to avoid opening the viewer during automated testing of this notebook
# You should ignore this!
if not AUTOMATED_TESTING:
    sv = SentenceNgramViewer(c.candidates[:100])
else:
    sv = None

And, now we render the `Viewer`:

In [ ]:
sv

Note that we can **navigate using the provided buttons**, or **using the keyboard (hover over buttons to see controls)**, highlight candidates (even if they overlap), and also **apply binary labels** (more on where to use this later!).  In particular, note that **the Viewer is synced dynamically with the notebook**, so that we can for example get the `id` of the candidate that is currently selected, this candidate object itself, as well as any labels we've applied.  Try it out!

In [ ]:
if not AUTOMATED_TESTING:
    print sv.selected_cid
    print sv.get_selected()
    print sv.get_labels()

## Composing a better candidate extractor

Now, let's try to increase our candidate recall using more of the `Matcher` operators and their functionalities.  First, let's turn on **Porter stemming** in our dictionary matcher; Porter stemming is an aggressive rules-based method for normalizing word endings.

In [ ]:
# Define a new matcher
matcher = DictionaryMatch(d=diseases, longest_match_only=False, stemmer='porter')

# Extract a new set of candidates
ce = CandidateExtractor(ngrams, matcher)
%time c = ce.extract(corpus.get_sentences())
#gold_stats(c, gold)

Next, note that *`Matcher` objects are compositional*. Observing in the `Viewer` that we are missing all of the acronyms, let's start with the `Union` operator, to integrate a dictionary for this:

In [ ]:
from snorkel.matchers import Union
from load_dictionaries import load_acronym_dictionary

# Load the disease phrase dictionary
acronyms = load_acronym_dictionary()
print "Loaded %s acronyms!" % len(acronyms)

# Define a new matcher
matcher = Union(
    DictionaryMatch(d=diseases, longest_match_only=False, stemmer='porter'),
    DictionaryMatch(d=acronyms, ignore_case=False))

# Extract a new set of candidates
ce = CandidateExtractor(ngrams, matcher)
%time c = ce.extract(corpus.get_sentences())
gold_stats(c, gold)

Next, we try using the `Concat` and `RegexMatch` operators to find candidate mentions composed of an _adjective followed by a term matching our diseases dictionary_.  Note in particular that we set `left_required=False` so that exact matches to our dictionary (with no adjective prepended) will still work:

In [ ]:
print corpus.get_sentences()[0].words
print corpus.get_sentences()[0].poses

In [ ]:
from snorkel.matchers import Concat, RegexMatchEach
matcher = Union(
    Concat(
        RegexMatchEach(rgx=r'JJ*', attrib='poses'),
        DictionaryMatch(d=diseases, longest_match_only=False, stemmer='porter'),
        left_required=False),
    DictionaryMatch(d=acronyms, ignore_case=False))

# Extract a new set of candidates
ce = CandidateExtractor(ngrams, matcher)
%time c = ce.extract(corpus.get_sentences())
#gold_stats(c, gold)

In [ ]:
matcher = DictionaryMatch(d=diseases, longest_match_only=False, stemmer='porter')
ce = CandidateExtractor(ngrams, matcher)
%time c = ce.extract(corpus.get_sentences())
c[:10]

In [ ]:
from snorkel.matchers import Concat, RegexMatchSpan

matcher = Concat(
    RegexMatchSpan(rgx=r'acute'),
    DictionaryMatch(d=diseases, longest_match_only=False, stemmer='porter'))
ce = CandidateExtractor(ngrams, matcher)
%time c = ce.extract(corpus.get_sentences())
c[:10]

In [ ]:
from snorkel.matchers import SlotFillMatch

matcher = SlotFillMatch(pattern="wt{0}-{1}", 
                        DictionaryMatch(d=proteins),
                        RegexMatchSpan(rgx="\d+"))

### Running candidate extraction in parallel

Note that **the candidate extraction procedure can be parallelized across multiple cores using the `parallelism=N`** optional argument to the `CandidateExtractor` object.

### More coming here...

We've increased the candidate recall (on the development set) by ~ 9% using some simple compositional `Matcher` operators.  We'll be adding more here soon!